# Peak Vermeidung durch Dispatching auf Rolling Mean

**Abstract**

In KNW-Opt sollen unterschiedliche Optimierungsmaßnahmen eines kalten Nahwärmenetzes untersucht werden.
Eine Maßnahme soll die elektrische Lastspitzenvermeidung des Wärmepumpenschwarms sein. Hierbei sollte möglichst
das gleichzeitige Anlaufen von vielen Wärmepumpenvermieden werden. Da die Wärmepumpen über Warmwasserspeicher verfügen, steht hier eine energetische Flexibilität zur Verfügung, welche eine gezielte Koordination der Wärmepumpen ermöglichen sollte, um Lastspitzen zu vermeiden.

##  Dispatching Algorithmus

###  Lastspitzen im KNW-Netz glätten

Eines der erklärten Ziele ist das Glätten elektrischer Lastspitzen im kalten Nahwäremenetz. Doch was soll Ausglätten bedeuten?
Ein möglichst glatter Betrieb der Wärmepumpen über einen diskreten Zeithorizont $T$ kann durch folgende *Kostenfunktion* beschrieben werden:

$$\min_{(x_{up},x_{down})} \sum_{t \in T} x_{up}(t) + x_{down}(t),$$

so dass folgende fünf Randbedingungen gelten:
 
$$P_{WP}(t) = \sum_{wp \in WP} P_{wp}(t),$$ 
$$mean(P_{WP}) = \frac{1}{|T|}\sum_{t \in T}P_{WP}(t),$$
$$P_{WP}(t) = mean(P_{WP}) + x_{up}(t) - x_{down}(t),$$
$$x_{up}(t), x_{down}(t)\geq 0$$


Hierbei ist $WP$ die Menge aller Wärmepumpen und $P_{wp}$ die (elektrische oder thermische) Leistung der Wärmepumpe $wp$. Mit anderen Worten, die Minimierungsaufgabe strebt den Baselastagang bei dem aggregierten Lastgang aller Wärmepumpen an:
Ist das Minimum $0$, so gilt nach (4) $x_{up}(t)=x_{down}(t)=0$ und dann mit (3) $P_{WP}(t) = mean(P_{WP})$ konstant.
Desweiteren muss auch noch folgende Bedingung gelten:

$P_{wp}(t)$ muss zu jedem Zeitpunkt $t$ die Wärmelast des Hauses, welches zur Wärmepumpe $wp$ gehört decken.

Warum ist das nun eine Lösung unseres Problems Lastspitzen zu vermeiden?

Wenn man die benötigte Energie als eine feste Zustandsgröße ansieht, so ist der Baselastgang derjenige Lastgang, welcher die geringste Leistungsspitzen aufweist. Denn jeder (stetige) Lastgang, welcher zu einer Zeit einen Leistungswert geringer als die Baselast aufweist, muss um die selbe Energie zu liefern zu einem anderen Zeitpunkt eine Leistung größer als die Baselast aufweisen. Und das ist gerade der Punkt: Der angestrebte Mittelwert bildet die Energie des Systems ab, das heißt unsere eigentliche Zielgröße ist die Baselast der benötigten Energie des Systems.

### Datengrundlage

Die Wärmepumpen haben eine Leistung von 1 - 8 kW und einen COP von 4.4 - 7.1. Der thermische Jahresenergiebedarf der Häuser beträgt 288 750 kWh.

Diese Parameter sind in der Excel Datei `210921_Zusammenfassung_der_WP_mit_Standort_Spieldaten_fuer_OpenSource.xlsx` zu finden.

Die Lastprofile wurden mit Hilfe der Datei `Loadprofiles_MAGGIE.csv` statistisch generiert.

Beide Datenblätter müssen angefragt werden.

Die Pufferspeicherkapazitäten zu Beginn der Simulation sind in [initial_levels_49_houses.csv](../data/initial_levels_49_houses.csv) zu finden.

### Flexibilitätsbetrachtungen im KNW-Netz

Zunächst soll eine Systemanalyse des KNW-Netz durch geführt werden. Diese Systemanalyse soll aufzeigen, in wie weit theoretisch eine Annäherung an den Baselastgang möglich ist. Im wesentlichen untersucht die Systemanalyse die von den Warmwasserspeichern zur Verfügung gestellte Flexibilität: Ist theoretisch eine Lastgangverschiebung der Wärmepumpen über einen Tag möglich, so dass ihr Summenlastgang dem Baselastgang entspricht?

Für die Systemanalyse modellieren wir das KNW-Netz als lineares Programm, welches obiges Optimierungsproblem formuliert, also diejenigen Lastgänge der Wärmepumpen sucht, so dass der Summenlastgang möglichst Nahe am Base liegen. Die Warmwasserspeicher werden also durch lineare Gleichungen modelliert, die Information der thermischen Verbräuche der Häuser stehen dem Optimierer in perfekter Voraussicht zur Verfügung und die Wärmepumpen können zwischen 0 und 100 Prozent moduliert werden. Es wird jeder Tag eines Jahres separat auf einem Stundenzeitgitter optimiert. Man beachte, dass dies noch keine Steuerungsstartegie liefert, welche diese Lastgänge in Laufzeit generieren könnte!  


Die Ergebins Lastgänge sehen wir hier [hier](./plots/ntp_oemof_solution_MAGGIE_1_day_steps_hepus_49.html).


*Von 365 Tagen war die Zielfunktion an 18 Tage ungleich null*, wenn Speicherstartzustände der Optimierung überlassen werden. An diesen Tagen konnte also der Tagesmittelwert der erwarteten elektrischen Leistung nicht eingehalten werden. Das heißt, dass die Ausregelung auf den Base in weniger als 5% der Tage nicht möglich ist.

Desweiteren skaliert diese Analyse in der Anzahl der Wärmepumpen rechenzeittechnisch bei Tages Simulationen auf einem Stundegitter linear:  


Prozessor		: AMD Ryzen 7 4700U with Radeon Graphics

Hauptspeicher		: 15657MB 


|Anzahl der Häuser|Rechenzeit in Sekunden|
|-----------------|----------------------|
|50|2093|
|100|5209|
|150|10748|
|200|15518|


### Algorithmische Flexibilisierung im KNW-Netz

Da die Analysen ergeben haben, dass eine Flexibilisierung auf den Base theoretisch sehr gut möglich ist, geht es in einem nächsten Schritt darum sich diesem Optimierungsziel algorithmisch in Laufzeit anzunähern. Das bedeutet, dass wir in Laufzeit zunächst dies benötigte Energie für 24 Stunden schätzen müssen. Diese Schätzung kann im einfachsten Fall über einen rollierenden Mittelwert der letzten 24 Stunden geschehen und liefert uns die *erwartete Baseleistung*. 

Alle Wärmepumpen sind mit Warmwasserspeicher ausgestattet und so haben sie eine gewisse Flexibilität in ihrem Betrieb. Genauer können wir die Flexibilität der Wärmepumpen über ihr *Potential* beschreiben. Das *Potential* ist definiert durch ihre maximale Leistung, momentane Leistung und der *Verfügbarkeit*. Eine Wärmepumpe ist *verfügbar*, wenn sich die Temperaturen im Warmwasserspeicher in enem Wohlfühlbereich befinden, etwa die mittlere Temperatur ist zwischen $T_{min}$ und  $T_{max}$, und die Wärmepumpe hat eine minimale Lauf- oder Standzeit durchlaufen, etwa eine Stunde. 

Zu jedem Zeitpunkt kann man nun die erwartete Baseleistung berechnen und das Potential aller Wärmepumpen abfragen. Die Leistung der nicht verfügbaren Wärmepumpen zieht man von der Baseleistung ab und die Restleistung verteilt man dann auf die verfügbaren Wärmepumpen. 

Dieses Verfahren nennen wir *Dispatching auf den Rolling Mean*.


###  Vorteile des Algorithmischen Modells

Der Algorithmus erstellt nicht für jede Wärmepumpe einen Einsatzplan, so dass der aggregierte Plan einen möglichst glatten Lastgang liefert, sondern er schätzt den idealen glatten Lastgang, den Base des Tages, und nutzt dann das ihm statistisch zur Verfügung stehende Potential um sich diesem Base anzunähern, bzw. die Leistung der Wärmepumpen so zu verteilen, dass ihre aggregierte Leistung diesem Base möglichst nahe kommt.

Der Vorteil gegenüber der indivuellen Einsatzplanung ist vor allem, dass keine Energien von Häusern und keine Energien in From von hochaufgelösten Zeitreihen Forecasts erstellt werden müssen. Dies ist wesentlich schwieriger und mit einem höheren Fehlerrisiko verbunden. Der Forecast der aggregierten Tagesenergie ist robuster und erfolgsversprechender, da statistische Protfolioeffekte vorhanden sind.

Desweiteren ist das algorithmishe Modell sparsam mit Daten und Berechnungen. Die Wärmepumpen sind als selbstständige Agenten modelliert, welche ihre individuellen Betriebsconstraints beliebig realisieren können. Dadurch ist die Optimierung des Wärmepumpenbetriebs im Sinnen der Wärmelieferung in den Häusern ermöglicht, ohne die individuellen Betriebsconstraints in der globalen Optimierung berücksichtigen zu müssen.

###  Simulation des Algorithmus 'Dispatching auf Rolling Mean' und Analyse des Algorithmus

Um den oben beschriebenen Algorithmus zu testen und zu überprüfen, ob er unser Ziel der Lastspitzenvermeidung realisiert, wurde eine Simualtionsumgebung entwickelt. Sie orientiert sich an Agenten basierten Simulationen, realisert aber Häuser und Leitzentrale nicht als selbstständige Software-Agenten/Programme.

Es handelt sich um eine Simulation von 49 Häusern mit thermischen Verbrauch. Diese besitzen jeweils eine Wärmepumpe und einen Pufferspeicher.

#### Daten

Die Wärmepumpen haben eine Leistung von 1 - 8 kW und einen COP von 4.4 - 7.1. Der thermische Jahresenergiebedarf der Häuser beträgt 288 750 kWh.

Diese Parameter sind in der Excel Datei `210921_Zusammenfassung_der_WP_mit_Standort_Spieldaten_fuer_OpenSource.xlsx` zu finden.

Die Lastprofile wurden mit Hilfe der Datei `Loadprofiles_MAGGIE.csv` statistisch generiert.

Beide Datenblätter müssen angefragt werden.

Die Pufferspeicherkapazitäten zu Beginn der Simulation sind in [initial_levels_49_houses.csv](../data/initial_levels_49_houses.csv) zu finden.

Die Ergebnisse einer klassischen Hysterese-Regelung sehen wir [hier](./plots/plot_simulation_no_dispatch_final_MAGGIE_49.html):

Mittlere aggregierte Leistung: 32.97 kW

Standardabweichung der aggregierten Leistung: 44.66 kW

Maximum der aggregierten Leistung: 424.7 kW

Minimum der aggregierten Leistung: 0 kW

90%-Qunatil der Aggregierten Leistung: 94.2 kW




Die Ergebnisse Regelung mit Dispatching auf den rolling Mean sehen wir [hier](./plots/plot_simulation_dispatch_final_MAGGIE_49.html):

Mittlere aggregierte Leistung: 32.99 kW

Standardabweichung der aggregierten Leistung: 28.91 kW

Maximum der aggregierten Leistung: 221.4 kW

Minimum der aggregierten Leistung: 0 kW

90%-Qunatil der Aggregierten Leistung: 74.8 kW


###  Kostenanalyse

Hierbei wurden die Kosten aus der Niederspannung-Preistabelle von [Bayernwerk (S.2)](https://www.bayernwerk-netz.de/content/dam/revu-global/bayernwerk-netz/files/netz/netzzugang/netzentgeltestrom/20201006-bayernwerk-preisblaetter-strom-2021gesamt.pdf) verwendet.

#### Flexibilitätsanalyse mit OEMOF

Vollbenutzungsstunden: 3 042.25 h

Maximale jährliche elektrische Leistung: 15.87 kW*a

Verrechnungswirkarbeit: 48 290.46 kWh

Kosten: $$15.87 \; \textrm{kW*a} \cdot 97.75 \frac{\textrm{€}}{\textrm{kW*a}} + 48 290.46 \; \textrm{kWh} \cdot 0.0175 \frac{\textrm{€}}{\textrm{kWh}} = 2 396.38 \textrm{€} $$

#### Dispatcher

Vollbenutzungsstunden: 1272.39 h

Maximale jährliche elektrische Leistung: 38.0 kW*a

Verrechnungswirkarbeit: 48 351 kWh

Kosten: $$38 \; \textrm{kW*a} \cdot 15.04 \frac{\textrm{€}}{\textrm{kW*a}} + 48 351 \; \textrm{kWh} \cdot 0.0505 \frac{\textrm{€}}{\textrm{kWh}} = 3 013.25 \textrm{€} $$

#### Hysterese

Vollbenutzungsstunden: 671.03 h

Maximale jährliche elektrische Leistung: 72 kW*a

Verrechnungswirkarbeit: 48 314 kWh

Kosten: $$72 \; \textrm{kW*a} \cdot 15.04 \frac{\textrm{€}}{\textrm{kW*a}} + 48 314 \; \textrm{kWh} \cdot 0.0505 \frac{\textrm{€}}{\textrm{kWh}} = 3 522.73 \textrm{€} $$

Durch den Dispatching Algorithmus wird also 509.49 € gegenüber dem Benchmarkszenario der Hystereseregelung eingespart.


###  Performance des Dispatching auf den Rolling Mean

Es wurde eine Performanzanalyse des Dispatching Algorithmus durchgeführt:


Prozessor		: AMD Ryzen 7 4700U with Radeon Graphics

Hauptspeicher		: 15657MB 


|number_of_houses|computation_time|steps|
|----------------|----------------|-----|
|5|0.374|24|
|25|0.607|24|
|125|1.416|24|
|625|5.648|24|
|3125|28.093|24|
|15625|141.250|24|

Man sieht, dass der Algorithmus für sehr große Wärmepumpenschwärme geeignet ist. 

###  Fazit Simulation

Die Simulationsergebnisse zeigen deutlich, dass durch Dispatching auf den Rolling Mean das Ziel der Peakvermeidung realisiert werden kanns wurde eine Reduktion der Peaklast um 22 kW bei 49 Häusern erreicht. Desweiteren stellt es eine Software technisch leicht umzusetzende Lösung dar, welche gut in der Größe der Wärmepumpenschwärme skaliert. Die Kostenersparnisse (509.49 €) durch die Peakvermeidung fallen gering aus, da der Lastgang eine geringe Anzahl von Jahresvollbenutzungsstunden aufweist und da der Leistungspreis nicht so stark ins Gewicht fällt. 


